Please do same exercise using two different framework.

Framework 1. pandas
framework 2. apache beam python https://beam.apache.org/documentation/sdks/python/


using two input files dataset1 and dataset2 

join dataset1 with dataset2 and get tier

generate below output file

legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)

Also create new record to add total for each of legal entity, counterparty & tier.

Sample data:
legal_entity, counterparty, tier, max(rating by counterparty), sum(value where status=ARAP), sum(value where status=ACCR)
L1,Total, Total, calculated_value, calculated_value,calculated_value
L1, C1, Total,calculated_value, calculated_value,calculated_value
Total,C1,Total,calculated_value, calculated_value,calculated_value
Total,Total,1,calculated_value, calculated_value,calculated_value
L2,Total,Total,calculated_value, calculated_value,calculated_value
....
like all other values.

where caluclated_value in sample data needs to be calculated using above method.

In [1]:
import pandas as pd

In [3]:
df_1 = pd.read_csv("../input_data/dataset1.csv")
df_2 = pd.read_csv("../input_data/dataset2.csv")

In [5]:
df_2

,counter_party,tier
0,C1,1
1,C2,2
2,C3,3
3,C4,4
4,C5,5
5,C6,6


In [6]:
df_1

,invoice_id,legal_entity,counter_party,rating,status,value
0,1,L1,C1,1,ARAP,10
1,2,L2,C2,2,ARAP,20
2,3,L3,C3,4,ACCR,30
3,4,L1,C4,6,ARAP,40
4,5,L2,C5,4,ACCR,50
5,6,L3,C6,6,ACCR,60
6,7,L1,C1,2,ARAP,10
7,8,L2,C2,3,ACCR,40
8,9,L3,C3,3,ACCR,80
9,10,L1,C4,5,ACCR,100


In [7]:
df_combined  = pd.merge(df_1, df_2, on='counter_party', how='left')
df_combined

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,2,L2,C2,2,ARAP,20,2
2,3,L3,C3,4,ACCR,30,3
3,4,L1,C4,6,ARAP,40,4
4,5,L2,C5,4,ACCR,50,5
5,6,L3,C6,6,ACCR,60,6
6,7,L1,C1,2,ARAP,10,1
7,8,L2,C2,3,ACCR,40,2
8,9,L3,C3,3,ACCR,80,3
9,10,L1,C4,5,ACCR,100,4


In [9]:
merged_df = df_1.merge(df_2, on='counter_party')
merged_df

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
1,7,L1,C1,2,ARAP,10,1
2,13,L1,C1,3,ARAP,20,1
3,2,L2,C2,2,ARAP,20,2
4,8,L2,C2,3,ACCR,40,2
5,3,L3,C3,4,ACCR,30,3
6,9,L3,C3,3,ACCR,80,3
7,14,L2,C3,2,ACCR,52,3
8,15,L3,C3,4,ACCR,35,3
9,16,L1,C3,6,ARAP,5,3


In [13]:
sorted_df = df_combined.sort_values(by='counter_party')

In [15]:
sorted_df

,invoice_id,legal_entity,counter_party,rating,status,value,tier
0,1,L1,C1,1,ARAP,10,1
12,13,L1,C1,3,ARAP,20,1
6,7,L1,C1,2,ARAP,10,1
1,2,L2,C2,2,ARAP,20,2
7,8,L2,C2,3,ACCR,40,2
15,16,L1,C3,6,ARAP,5,3
14,15,L3,C3,4,ACCR,35,3
13,14,L2,C3,2,ACCR,52,3
8,9,L3,C3,3,ACCR,80,3
2,3,L3,C3,4,ACCR,30,3


In [23]:
grouped_df = df_combined.groupby(['legal_entity', 'counter_party', 'tier'])

# Define a function to apply to each group
def calculate_values(group):
    rating = group['rating'].max()
    arap_sum = group.loc[group['status'] == 'ARAP', 'value'].sum()
    accr_sum = group.loc[group['status'] == 'ACCR', 'value'].sum()
    return pd.Series({'max(rating by counterparty)': rating, 'sum(value where status=ARAP)': arap_sum, 'sum(value where status=ACCR)': accr_sum})

# Apply the function to each group and create a new dataframe
agg_df = grouped_df.apply(calculate_values)
result_df = agg_df.reset_index()

In [24]:
result_df

,legal_entity,counter_party,tier,max(rating by counterparty),sum(value where status=ARAP),sum(value where status=ACCR)
0,L1,C1,1,3,40,0
1,L1,C3,3,6,5,0
2,L1,C4,4,6,40,100
3,L2,C2,2,3,20,40
4,L2,C3,3,2,0,52
5,L2,C5,5,6,1000,115
6,L3,C3,3,4,0,145
7,L3,C6,6,6,145,60


In [25]:
legal_entity_total = result_df.groupby('legal_entity').agg({'rating': 'max', 
                                                            'value': lambda x: x.sum()}).reset_index()
legal_entity_total['counter_party'] = 'Total'
legal_entity_total['tier'] = 'Total'

KeyError: "Column(s) ['rating', 'value'] do not exist"

In [26]:
# Group the data by legal_entity
grouped_df = df_combined.groupby(['legal_entity'])

# Define a function to apply to each group
def calculate_values(group):
    rating = group['max(rating by counterparty)'].max()
    arap_sum = group['sum(value where status=ARAP)'].sum()
    accr_sum = group['sum(value where status=ACCR)'].sum()
    return pd.Series({'max(rating by counterparty)': rating, 'sum(value where status=ARAP)': arap_sum, 'sum(value where status=ACCR)': accr_sum})

# Apply the function to each group and create a new dataframe
legal_entity_total = grouped_df.apply(calculate_values)

# Add the 'legal_entity' column to the dataframe
legal_entity_total['legal_entity'] = legal_entity_total.index

# Set the 'counter_party' and 'tier' columns to 'Total'
legal_entity_total['counter_party'] = 'Total'
legal_entity_total['tier'] = 'Total'

# Reorder the columns
legal_entity_total = legal_entity_total[['legal_entity', 'counter_party', 'tier', 'max(rating by counterparty)', 'sum(value where status=ARAP)', 'sum(value where status=ACCR)']]


KeyError: 'max(rating by counterparty)'